In [ ]:
## CHARGEMENT DES BIBLIOTHEQUES
############################################################

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import geopandas as gpd
import plotly.express as px

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import pearsonr
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [ ]:
## FONCTIONS UTILES
############################################################

##
# CHARGEMENT DE FICHIERS JSON

# Fonction qui permet de charger un fichier json dans un dictionnaire
def load_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data


##
# CHARGEMENT DU DATAFRAME

# On charge les données dans un dataframe "df"
def load_dataset(path):
    df = pd.read_csv(path)

    return df


##
# FILTRAGE DE COLONNES

# On récupère ici seulement les évènements qui nous intéressent, ceux liés à des violences politiques
# On conserve 'Strategic developments' car elle permet de capturer le contexte
def columns_filtering(df):
    events_filter = ['Battles', 'Explosions/Remote violence',
                    'Violence against civilians', 'Strategic developments']

    df = df[df['event_type'].isin(events_filter)]
    
    return df


##
# CREATION DE NOUVELLES COLONNES

def add_new_colums_dates(df):
    ##
    # On transforme la colonne "event_date" en datetime pour la manipuler plus facilement
    df['event_date'] = pd.to_datetime(df['event_date'])

    # On crée la colonne "month" en utilisant l'attribut month de datetime
    df['month'] = df['event_date'].dt.month
    # On ajoute la colonne juste après "year"
    df.insert(3, 'month', df.pop('month'))

    # On crée la colonne "day" en utilisant l'attribut day de datetime
    df['day'] = df['event_date'].dt.day
    # On ajoute la colonne juste après "month"
    df.insert(5, 'day', df.pop('day'))
    
    return df


def add_new_colums_actors(df):
    ##
    # On charge les données du fichier json "actor_type" dans un dictionnaire
    # qui associe les valeurs des colonnes "inter1" et "inter2"
    # au nom de chaque catégorie d'acteur (source : codebook ACLED)
    actor_type = load_json_file("../references/actor_type.json")

    # On convertit les clés du dictionnaire en entiers
    actor_type = {int(k): v for k, v in actor_type.items()}

    # On ajoute les colonnes "actor1_type" et "actor2_type" au dataframe
    df['actor1_type'] = df['inter1'].map(actor_type)
    df['actor2_type'] = df['inter2'].map(actor_type)

    return df


def add_new_colums_interaction(df):
    ##
    # On charge les données du fichier json "interaction_type" dans un dictionnaire
    # associant aux valeurs de la colonne "interaction"
    # les 2 acteurs impliqués dans une confrontation (source : codebook ACLED)
    interaction_type = load_json_file("../references/interaction_type.json")

    # On convertit les clés du dictionnaire en entiers
    interaction_type = {int(k): v for k, v in interaction_type.items()}

    # On ajoute une colonne "interaction_type" au dataframe
    df['interaction_type'] = df['interaction'].map(interaction_type)

    return df


def add_new_colums_terrorist_group(df):
    # On crée une nouvelle colonne qui indique si pour un évènement l'un des
    # acteurs est une organisation terroriste
    # On charge les données du fichier json "terrorist_group_filiation"
    # dans un dictionnaire associant organisation terroriste et organisation mère
    terrorist_group_filiation = load_json_file("../references/terrorist_group_filiation.json")

    # On crée une liste contenant les organisations terroristes à partir des indices de ce dictionnaire
    terrorist_groups = list(terrorist_group_filiation.keys())

    # On crée une fonction pour vérifier si un acteur est une organisation terroriste
    def is_terrorist_actor(actor):
        return actor in terrorist_groups

    # On ajoute la colonne "is_terrorist_group_related" au dataframe
    df['is_terrorist_group_related'] = (df['actor1'].apply(is_terrorist_actor) |
                                df['assoc_actor_1'].apply(is_terrorist_actor) |
                                df['actor2'].apply(is_terrorist_actor) |
                                df['assoc_actor_2'].apply(is_terrorist_actor)).astype(int)

    # On crée une fonction de mapping pour associer les valeurs du dictionnaire aux acteurs
    # S'il n'y a pas de valeur on retourne "None" car cela veut simplement dire que l'évènement
    # n'est pas lié à une organisation terroriste et qu'il n'y a donc pas de lien de filiation
    # avec une organisation mère
    def map_filiation(row):
        for actor in ['actor1', 'assoc_actor_1', 'actor2', 'assoc_actor_2']:
            if row[actor] in terrorist_group_filiation:
                return terrorist_group_filiation[row[actor]]
        return "None"

    # On ajoute la colonne "terrorist_group_filiation" au dataframe
    df['terrorist_group_filiation'] = df.apply(map_filiation, axis=1)

    return df


def add_new_colums_pmc_group(df):
    # On crée une nouvelle colonne qui indique si pour un évènement l'un
    # des acteurs est une pmc russe
    pmc_groups = ['Wagner Group']

    # On crée une fonction pour vérifier si un acteur est une pmc russe
    def is_pmc_actor(actor):
        return actor in pmc_groups

    # On ajoute la colonne "is_pmc_related" au dataframe
    df['is_pmc_related'] = (df['actor1'].apply(is_pmc_actor) |
                                df['assoc_actor_1'].apply(is_pmc_actor) |
                                df['actor2'].apply(is_pmc_actor) |
                                df['assoc_actor_2'].apply(is_pmc_actor)).astype(int)
    
    return df


##
# SUPPRESSION DE COLONNES

def delete_columns(df):
    # On supprime les colonnes qui ne nous serons d'aucune utilité
    columns_to_drop = [
        'time_precision',
        'disorder_type',
        'sub_event_type',
        'actor1',
        'assoc_actor_1',
        'actor2',
        'assoc_actor_2',
        'inter1',
        'inter2',
        'interaction',
        'admin2',
        'admin3',
        'iso',
        'region',
        'location',
        'latitude',
        'longitude',
        'geo_precision',
        'source',
        'source_scale',
        'notes',
        'tags',
        'timestamp',
        'civilian_targeting',
        'event_id_cnty'
    ]

    df = df.drop(columns=columns_to_drop, axis=1)

    return df


##
# GESTION DES VALEURS MANQUANTES

def fill_nan_values(df):
    # Pour la colonne "actor2_type" on va remplacer les valeurs manquantes par "None" car cela
    # indique juste que dans certains cas qu'il y a un seul acteur et pas un manque de valeur
    df['actor2_type'] = df['actor2_type'].fillna("None")

    return df


##
# Fonction pour mapper les noms de sous-régions aux identifiants de coordonnées
def map_regions_to_ids(data, mapping):
    data['region_id'] = data['admin1'].map(mapping)
    return data

In [ ]:
## CONSTRUCTION DU DATAFRAME HISTORIQUE "past_data"
############################################################

def build_past_data(df):
    # On compte les occurences de chaque valeur des colonnes 'actor1_type' et 'actor2_type'
    actor_counts = df.melt(id_vars=['event_date', 'country', 'admin1', 'year', 'month', 'is_terrorist_group_related', 'fatalities', 'event_type'], 
                        value_vars=['actor1_type', 'actor2_type'], 
                        var_name='actor_role', value_name='actor')

    actor_counts = actor_counts.groupby(['country', 'admin1', 'year', 'month', 'actor']).size().unstack(fill_value=0).reset_index()

    # On compte les occurences de chaque type d'évènement
    event_type_counts = df.pivot_table(index=['country', 'admin1', 'year', 'month'], 
                                    columns='event_type', 
                                    aggfunc='size', 
                                    fill_value=0).reset_index()

    # On agrège les données par mois, par pays, par sous-région
    past_data = df.groupby(['country', 'admin1', 'year', 'month']).agg(
        total_events=('event_date', 'count'),
        terrorist_events=('is_terrorist_group_related', 'sum'),
        fatalities=('fatalities', 'sum')
    ).reset_index()

    # On fusionne les variables des acteurs et des types d'évènements avec les données agrégées
    past_data = past_data.merge(actor_counts, on=['country', 'admin1', 'year', 'month'], how='left')
    past_data = past_data.merge(event_type_counts, on=['country', 'admin1', 'year', 'month'], how='left')

    # On supprime les colonnes 'None' et 'Protesters' qui concernent les acteurs mais sont sans intérêt pour notre étude
    # ou bien la colonne 'Rebel Groups' qui est dérivée de la variable cible
    past_data = past_data.drop(columns=['None', 'Protesters', 'Rebel Groups'], axis=1)

    # On crée des features décalées (lags) pour capturer les tendances passées des événements totaux et des événements terroristes
    for admin in past_data['admin1'].unique():  # Pour chaque valeur unique dans la colonne 'admin1'
        
        for lag in range(1, 7):  # Pour chaque décalage (lag) de 1 à 6
            lag_total_events = past_data[past_data['admin1'] == admin]['total_events'].shift(lag)  # Crée une série décalée pour 'total_events' pour l'admin actuel
            lag_terrorist_events = past_data[past_data['admin1'] == admin]['terrorist_events'].shift(lag)  # Crée une série décalée pour 'terrorist_events' sans filtrer par admin
            past_data.loc[past_data['admin1'] == admin, f'total_events_lag_{lag}'] = lag_total_events  # Ajoute la série décalée pour 'total_events' dans une nouvelle colonne pour l'admin actuel
            past_data.loc[past_data['admin1'] == admin, f'terrorist_events_lag_{lag}'] = lag_terrorist_events  # Ajoute la série décalée pour 'terrorist_events' dans une nouvelle colonne pour l'admin actuel
            past_data[[f'total_events_lag_{lag}', f'terrorist_events_lag_{lag}']] = past_data[
                [f'total_events_lag_{lag}', f'terrorist_events_lag_{lag}']
                ].fillna(0)  # Remplit les valeurs manquantes dans les colonnes décalées avec des zéros

    # On supprime les valeurs NA générées par les lags
    # past_data.dropna(inplace=True)  # Optionnel : supprime les lignes contenant des valeurs NA (commenté ici)

    # On supprime la colonne "total_events" qui est trop corrélée à notre variable cible
    past_data = past_data.drop(columns=['total_events'], axis=1)

    return past_data


In [ ]:
## TRANSFORMATION DES DONNEES DU DATAFRAME "past_data"
############################################################

def past_data_transformation(df):

    ##
    # On trie notre dataframe par année puis mois dans l'ordre croissant
    # pour que les données de test concernent les événements les plus récents
    df = df.sort_values(by=['year', 'month'])

    # Séparation des variables explicatives de notre variable cible
    X = df.drop(columns='terrorist_events', axis=1)
    y = df['terrorist_events']

    # Division en ensembles d'entraînement et de test avec shuffle à False
    # pour que les données de test concernent les événements les plus récents (6 derniers mois)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

    # On sauvegarde une version non encodée de X_test
    X_test_no_encoding = X_test


    ##
    # On standardise les variables numériques exceptées les variables catégorielles
    # ainsi que l'année et le mois qui subiront une autre transformation
    cat_columns=['country', 'admin1']
    X_train_numerical_vars = [col for col in X_train.columns if col not in cat_columns + ['year', 'month']]
    X_test_numerical_vars = [col for col in X_test.columns if col not in cat_columns + ['year', 'month']]
    # Extraction des données numériques
    X_train_numerical = X_train[X_train_numerical_vars].values
    X_test_numerical = X_test[X_test_numerical_vars].values

    # Initialisation d'un scaler
    scaler = StandardScaler()
    X_train[X_train_numerical_vars] = scaler.fit_transform( X_train_numerical)
    X_test[X_test_numerical_vars] = scaler.transform(X_test_numerical)


    ##
    # Hot One Encoding des variables catégorielles
    X_train_encoded = pd.get_dummies(X_train, columns=cat_columns, dtype=int)
    X_test_encoded = pd.get_dummies(X_test, columns=cat_columns, dtype=int)
    # On aligne les colonnes de X_test_encoded sur celles de X_train_encoded
    # pour s'assurer qu'ils ont les mêmes colonnes
    X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)


    ##
    # Transformation trigonométrique de 'year' et 'month' pour la saisonnalité
    X_train_encoded['month_sin'] = np.sin(2 * np.pi * X_train_encoded['month'] / 12)
    X_train_encoded['month_cos'] = np.cos(2 * np.pi * X_train_encoded['month'] / 12)
    X_train_encoded['year_sin'] = np.sin(2 * np.pi * (X_train_encoded['year'] - X_train_encoded['year'].min()) / 
                                    (X_train_encoded['year'].max() - X_train_encoded['year'].min() + 1))
    X_train_encoded['year_cos'] = np.cos(2 * np.pi * (X_train_encoded['year'] - X_train_encoded['year'].min()) / 
                                    (X_train_encoded['year'].max() - X_train_encoded['year'].min() + 1))
    
    X_test_encoded['month_sin'] = np.sin(2 * np.pi * X_test_encoded['month'] / 12)
    X_test_encoded['month_cos'] = np.cos(2 * np.pi * X_test_encoded['month'] / 12)
    X_test_encoded['year_sin'] = np.sin(2 * np.pi * (X_test_encoded['year'] - X_test_encoded['year'].min()) / 
                                    (X_test_encoded['year'].max() - X_test_encoded['year'].min() + 1))
    X_test_encoded['year_cos'] = np.cos(2 * np.pi * (X_test_encoded['year'] - X_test_encoded['year'].min()) / 
                                    (X_test_encoded['year'].max() - X_test_encoded['year'].min() + 1))

    # Suppression des colonnes 'month' et 'year' après transformation
    X_train_encoded = X_train_encoded.drop(columns=['month', 'year'])
    X_test_encoded = X_test_encoded.drop(columns=['month', 'year'])

    return X_train_encoded, X_test_encoded, X_test_no_encoding, y_train, y_test, scaler

In [ ]:
## CONSTRUCTION DU DATAFRAME FUTUR "future_data"
## POUR LES PREDICTIONS DES 6 PROCHAINS MOIS
############################################################

def build_future_data(past_data):
    unique_country_admin1 = past_data[['country', 'admin1']].drop_duplicates()

    # On souhaite d'abord obtenir le dernier mois de la dernière l'année dans past_data
    last_year = past_data['year'].max()
    last_month = past_data[past_data['year'] == last_year]['month'].max()

    # On crée ensuite une liste de dictionnaires pour les 6 prochains mois avec les colonnes explicatives vides
    future_data_list = []

    # On ajoute les 6 prochains mois à future_data_list avec les colonnes connues "country" et "admin1" remplies
    for i in range(1, 7):
        next_month = last_month + i
        next_year = last_year
        if next_month > 12:
            next_month -= 12
            next_year += 1
        for _, row in unique_country_admin1.iterrows():
            future_data_list.append({
                'year': next_year, 
                'month': next_month, 
                'country': row['country'], 
                'admin1': row['admin1']
            })

    # On crée notre dataframe "future_data" à partir de la liste de dictionnaires
    future_data = pd.DataFrame(future_data_list, columns=past_data.columns)

    # On remplit les autres colonnes explicatives avec des valeurs manquantes
    for col in past_data.columns:
        if col not in ['year', 'month', 'country', 'admin1']:
            future_data[col] = np.nan

    # Liste des colonnes pour lesquelles on veut remplir les valeurs manquantes
    # columns_to_fill = [
    #     'total_events', 'fatalities', 'Civilians', 'External/Other Forces',
    #     'Identity Militias', 'Political Militias',
    #     'State Forces', 'Battles', 'Explosions/Remote violence',
    #     'Strategic developments', 'Violence against civilians',
    #     'total_events_lag_1', 'terrorist_events_lag_1',
    #     'total_events_lag_2', 'terrorist_events_lag_2',
    #     'total_events_lag_3', 'terrorist_events_lag_3',
    #     'total_events_lag_4', 'terrorist_events_lag_4',
    #     'total_events_lag_5', 'terrorist_events_lag_5',
    #     'total_events_lag_6', 'terrorist_events_lag_6'
    # ]

    columns_to_fill = [
        'fatalities', 'Civilians', 'External/Other Forces',
        'Identity Militias', 'Political Militias',
        'State Forces', 'Battles', 'Explosions/Remote violence',
        'Strategic developments', 'Violence against civilians',
        'total_events_lag_1', 'terrorist_events_lag_1',
        'total_events_lag_2', 'terrorist_events_lag_2',
        'total_events_lag_3', 'terrorist_events_lag_3',
        'total_events_lag_4', 'terrorist_events_lag_4',
        'total_events_lag_5', 'terrorist_events_lag_5',
        'total_events_lag_6', 'terrorist_events_lag_6'
    ]


    # On groupe les données par 'country', 'admin1', 'month' pour calculer les moyennes dans past_data
    grouped_data = past_data.groupby(['country', 'admin1', 'month']).mean().reset_index()

    # On remplace les valeurs manquantes dans future_data par les moyennes correspondantes de past_data
    # en fonction du mois. Ex : si on veut calculer les valeurs des variables du mois de juillet 2024,
    # on va calculer la moyenne des variables des mois de juillet des années précédentes pour avoir des
    # valeurs de test réalistes.
    for index, row in future_data.iterrows():
        country = row['country']
        admin1 = row['admin1']
        month = row['month']
        for var in columns_to_fill:
            mean_value = grouped_data[(grouped_data['country'] == country) & 
                                    (grouped_data['admin1'] == admin1) & 
                                    (grouped_data['month'] == month)][var].values
            if len(mean_value) > 0:
                future_data.at[index, var] = mean_value[0]

    # Certaines valeurs sont manquantes car les combinaisons (country, admin1, month) n'existent pas
    # on va donc créer un dataframe contenant les combinaisons uniques de (country, admin1, month) dans future_data
    future_combinations = future_data[['country', 'admin1', 'month']].drop_duplicates()

    # Puis un dataframe contenant les combinaisons uniques de (country, admin1, month) dans past_data
    monthly_combinations = past_data[['country', 'admin1', 'month']].drop_duplicates()

    # Ensuite on recherche les combinaisons présentes dans future_data mais absentes dans past_data
    missing_combinations = future_combinations[~future_combinations.isin(monthly_combinations)].dropna()

    # Affichage des combinaisons manquantes
    # print("Combinaisons manquantes dans past_data :\n"missing_combinations)

    # En fonction de ces combinaisons, on vient remplir les valeurs manquantes dans future_data
    # à partir du calcul de la moyenne des autres mois pour la combinaison (country, admin1)
    for index, row in missing_combinations.iterrows():
        # Sélection des lignes correspondant à la combinaison de country et admin1 dans past_data
        matching_rows = past_data[(past_data['country'] == row['country']) & (past_data['admin1'] == row['admin1'])]
        # Calcul de la moyenne des valeurs des variables pour tous les mois existants
        mean_values = matching_rows[columns_to_fill].mean()
        # Remplissage des valeurs manquantes dans future_data avec la moyenne calculée
        future_data.loc[(future_data['country'] == row['country']) & (future_data['admin1'] == row['admin1']) & (future_data['month'] == row['month']), columns_to_fill] = mean_values.values

    # Enfin on remplace les NaN des lignes restantes par la valeur la plus fréquente dans chaque colonne
    future_data = future_data.fillna(future_data.mode().iloc[0])

    # On supprime la colonne de variable cible et on vérifie qu'il n'y a plus de NaN
    future_data = future_data.drop(columns=['terrorist_events'])
    future_data.info()

    # On arrondit les valeurs calculées pour les variables explicatives afin d'obtenir des valeurs entières
    future_data = future_data.round(0)

    return future_data

In [ ]:
## TRANSFORMATION DES DONNEES DU DATAFRAME "future_data"
############################################################

def future_data_transformation(df, scaler):
    ##
    # Préparation du dataframe "future_data" pour le modèle de ML
    future_data_encoded = df.copy()

    # On standardise les variables numériques exceptées les variables catégorielles
    # ainsi que l'année et le mois qui subiront une autre transformation
    cat_columns=['country', 'admin1']
    future_data_numerical_vars = [col for col in future_data_encoded.columns if col not in cat_columns + ['year', 'month']]
    # Extraction des données numériques
    future_data_numerical = df[future_data_numerical_vars].values

    # On utilise le scaler ajusté sur les données d'entraînement pour transformer les données du dataframe
    future_data_encoded[future_data_numerical_vars] = scaler.transform(future_data_numerical)

    # Hot One Encoding des variables catégorielles
    future_data_encoded = pd.get_dummies(future_data_encoded, columns=cat_columns, dtype=int)

    # Transformation trigonométrique de 'year' et 'month' pour la saisonnalité
    future_data_encoded['month_sin'] = np.sin(2 * np.pi * future_data_encoded['month'] / 12)
    future_data_encoded['month_cos'] = np.cos(2 * np.pi * future_data_encoded['month'] / 12)
    future_data_encoded['year_sin'] = np.sin(2 * np.pi * (future_data_encoded['year'] - future_data_encoded['year'].min()) / 
                                    (future_data_encoded['year'].max() - future_data_encoded['year'].min() + 1))
    future_data_encoded['year_cos'] = np.cos(2 * np.pi * (future_data_encoded['year'] - future_data_encoded['year'].min()) / 
                                    (future_data_encoded['year'].max() - future_data_encoded['year'].min() + 1))

    # Suppression des colonnes 'month' et 'year' après transformation
    future_data_encoded = future_data_encoded.drop(columns=['month', 'year'])

    return future_data_encoded

In [ ]:
## CREATION DES DATAFRAMES HISTORIQUES ET FUTURS
############################################################

# On applique les différentes étapes de préparation à notre dataframe initial
df = load_dataset('../data/raw/terrorisme_sahel.csv')
df = columns_filtering(df)
df = add_new_colums_dates(df)
df = add_new_colums_actors(df)
df = add_new_colums_interaction(df)
df = add_new_colums_terrorist_group(df)
df = add_new_colums_pmc_group(df)
df = delete_columns(df)
df = fill_nan_values(df)

# On crée notre dataframe qui sera utilisé pour entrainer notre modèle
# sur les données historiques
past_data = build_past_data(df)

# On crée notre dataframe qui sera utilisé pour effectuer les prédictions
# sur les 6 prochains mois
future_data = build_future_data(past_data)

In [ ]:
## PREDICTION POUR LES 6 PROCHAINS MOIS
## AVEC UN MODELE RANDOM FOREST REGRESSOR
## AVEC LDA
############################################################

# On récupère nos jeux d'entrainement et de test
X_train_encoded, X_test_encoded, X_test_no_encoding, y_train, y_test, scaler = past_data_transformation(past_data)
future_data_encoded = future_data_transformation(future_data, scaler)

lda = LDA()
X_train_lda = lda.fit_transform(X_train_encoded, y_train)
X_test_lda = lda.transform(X_test_encoded)
future_data_encoded_lda = lda.transform(future_data_encoded)

# On entraine un modèle RF avec la LDA
# avec les meilleurs paramètres du Gridsearch
rf = RandomForestRegressor(n_estimators=100, random_state=42)

rf.fit(X_train_lda, y_train)

# Prédictions de la LDA
y_pred_past_lda = rf.predict(X_test_lda)

# Évaluation de la LDA sur les données historiques
r2_lda = rf.score(X_test_lda, y_test)
rmse_lda = np.sqrt(mean_squared_error(y_test, y_pred_past_lda))
mae_lda = mean_absolute_error(y_test, y_pred_past_lda)
pearson_corr_lda, _ = pearsonr(y_test, y_pred_past_lda)

print("\nRandom Forest LDA R2:", r2_lda)
print("Random Forest LDA RMSE:", rmse_lda)
print("Random Forest LDA MAE:", mae_lda)
print("Random Forest LDA Pearson Correlation:", pearson_corr_lda)

# On utilise notre modèle pour faire des prédictions sur future_data
y_pred_future_lda = rf.predict(future_data_encoded_lda)

# On ajoute les prédictions à notre dataframe future_data
future_data['terrorist_events'] = y_pred_future_lda.round(0)

In [ ]:
coefficients = pd.DataFrame({"Feature":  X_train_encoded.columns, "Coefficient": lda.coef_[0]})
# Visualisation des coefficients
plt.figure(figsize=(10, 15))
plt.barh(coefficients['Feature'], sorted(coefficients['Coefficient']))
plt.xlabel('Coefficient Value')
plt.ylabel('Feature')
plt.title('Coefficients des features pour le modèle RF LDA')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# visualisation des résidus entre les prédictions et le réel pour les 6 derniers mois de nos data
residu = y_pred_past_lda - y_test
#plt.hist(residu, bins=25)
sns.histplot(residu, kde=True)
plt.xlabel('résidus= y_pred_past_lda - y_test')
plt.title('Histogramme des résidus')
_ = plt.ylabel('Count')
# hétéroscédasticité des résidus, mais ce n'est pas rédibitoire pour l'utilisation du modéle, sachant les bons résultats des métrics ci-dessus.

In [ ]:
# observation de la normalité de l'erreur
import statsmodels.api as sm
sm.qqplot(residu, fit = True, line = '45')
plt.title("Etude de la normalité des résidus")
plt.show();
# Les résidus pourraient s'approcher d'une loi normale entre -2 et 2, mais décroche qu-dessous de -2

In [ ]:
##
# On souhaite ajouter une colonne 'terrorist_events_last_6_months' à 'future_data' qui contient
# la moyenne de 'terrorist_events' des 6 derniers mois pour chaque mois et sous-région

# Convertir 'year' et 'month' en une seule colonne datetime pour faciliter le tri
past_data['date'] = pd.to_datetime(past_data[['year', 'month']].assign(day=1))
future_data['date'] = pd.to_datetime(future_data[['year', 'month']].assign(day=1))

# Trier les données par 'admin1' et 'date'
past_data = past_data.sort_values(by=['admin1', 'date'])
future_data = future_data.sort_values(by=['admin1', 'date'])

# Calculer la moyenne mobile sur les 6 derniers mois dans past_data
past_data['terrorist_events_last_6_months'] = past_data.groupby('admin1')['terrorist_events'].transform(lambda x: x.rolling(window=6, min_periods=1).mean().round())

# Obtenir les dernières valeurs de 'terrorist_events_last_6_months' pour chaque 'admin1'
last_6_months_avg = past_data.groupby('admin1').apply(lambda x: x.set_index('date').resample('M').last().ffill().iloc[-1]['terrorist_events_last_6_months'])

# Ajouter cette information à future_data
future_data['terrorist_events_last_6_months'] = future_data['admin1'].map(last_6_months_avg)


In [ ]:
# Visualisation des prédictions par rapport au réel
# par mois de Novembre 2023 à Avril 2024
# au Mali, au Burkina Faso et au Niger
# des les événements liés à des organisations terroristes

# On ajoute les prédictions à l'ensemble de test
df_test = X_test_no_encoding.copy()
df_test['terrorist_events'] = y_test
df_test['terrorist_events_pred'] = y_pred_past_lda

# On convertit 'year' et 'month' en une colonne datetime 'event_date'
df_test['event_date'] = pd.to_datetime(df_test[['year', 'month']].assign(day=1))

# On regroupe les événements réels par mois
real_events_by_month = df_test.groupby('event_date')['terrorist_events'].sum()

# On regroupe les événements prédits par mois
pred_events_by_month = df_test.groupby('event_date')['terrorist_events_pred'].sum()

# On crée notre graphique
plt.figure(figsize=(20, 8))
plt.plot(real_events_by_month.index, real_events_by_month.values, label='réel', linestyle='-', marker='o', color='blue')
plt.plot(pred_events_by_month.index, pred_events_by_month.values, label='prédit', linestyle='--', marker='x', color='orange')
plt.legend()
plt.title("Evénements liés à des organisations terroristes, par mois de Novembre 2023 à Avril 2024, au Mali, au Burkina Faso et au Niger")
plt.xlabel('Mois')
plt.ylabel("Nombre d'événements")
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%b %Y'))
plt.grid(True)
plt.tight_layout()
plt.show();

In [ ]:
import matplotlib.patches as mpatches
import random

all_data = pd.concat([past_data, future_data], axis=0, ignore_index=True)

# Filtrer les données pour le Mali
mali_all_data = all_data.loc[all_data['country'] == 'Mali']

# Grouper les données filtrées par 'year', 'month', 'country' et 'admin1'
# et calculer la somme des 'terrorist_events'
mali_terrorist_events_by_admin = mali_all_data.groupby(['year', 'month', 'country', 'admin1'])['terrorist_events'].sum().reset_index()

mali_terrorist_events_by_admin['date'] = pd.to_datetime(mali_terrorist_events_by_admin[['year', 'month']].assign(day=1))
mali_terrorist_events_by_admin.set_index('date', inplace=True)
# Visualisation des événements prédits
number_of_colors = 15

colors = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
             for i in range(number_of_colors)]

dict_color = dict(zip(mali_all_data['admin1'].unique(), colors))
legend_pops = []
plt.figure(figsize=(20, 8))
for admin in mali_all_data['admin1'].unique():
    pred_by_admin = mali_terrorist_events_by_admin[mali_terrorist_events_by_admin['admin1'] == admin]
    mali_pred_by_admin= pred_by_admin[(pred_by_admin['month'] >4) & (pred_by_admin['year'] == 2024)]
    mali_past_by_admin= pred_by_admin[~((pred_by_admin['month'] >4) & (pred_by_admin['year'] == 2024))]
    
    plt.plot(mali_past_by_admin.index, mali_past_by_admin['terrorist_events'], linestyle='-', marker='x', color=dict_color[admin])
    plt.plot(mali_pred_by_admin.index, mali_pred_by_admin['terrorist_events'], linestyle='--', marker='x', color=dict_color[admin])
    legend_pops.append(mpatches.Patch(color=dict_color[admin], label=admin)) 

plt.legend(handles=legend_pops) 

plt.title("Evénements passés et prédits liés à des organisations terroristes, par mois de juin 2021 à Octobre 2024 au Mali, par administration territoriale")
plt.xlabel('Mois')
plt.ylabel("Nombre d'événements par admin")
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%b %Y'))
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Filtrer les données pour le Burkina Faso
bf_all_data = all_data.loc[all_data['country'] == 'Burkina Faso']

# Grouper les données filtrées par 'year', 'month', 'country' et 'admin1'
# et calculer la somme des 'terrorist_events'
bf_terrorist_events_by_admin = bf_all_data.groupby(['year', 'month', 'country', 'admin1'])['terrorist_events'].sum().reset_index()

bf_terrorist_events_by_admin['date'] = pd.to_datetime(bf_terrorist_events_by_admin[['year', 'month']].assign(day=1))
bf_terrorist_events_by_admin.set_index('date', inplace=True)
# Visualisation des événements prédits
number_of_colors = 15

colors = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
             for i in range(number_of_colors)]

dict_color = dict(zip(bf_all_data['admin1'].unique(), colors))
legend_pops = []
plt.figure(figsize=(20, 8))
for admin in bf_all_data['admin1'].unique():
    pred_by_admin = bf_terrorist_events_by_admin[bf_terrorist_events_by_admin['admin1'] == admin]
    bf_pred_by_admin= pred_by_admin[(pred_by_admin['month'] >4) & (pred_by_admin['year'] == 2024)]
    bf_past_by_admin= pred_by_admin[~((pred_by_admin['month'] >4) & (pred_by_admin['year'] == 2024))]
    
    plt.plot(bf_past_by_admin.index, bf_past_by_admin['terrorist_events'], linestyle='-', marker='x', color=dict_color[admin])
    plt.plot(bf_pred_by_admin.index, bf_pred_by_admin['terrorist_events'], linestyle='--', marker='x', color=dict_color[admin])
    legend_pops.append(mpatches.Patch(color=dict_color[admin], label=admin)) 

plt.legend(handles=legend_pops) 

plt.title("Evénements passés et prédits liés à des organisations terroristes, par mois de juin 2021 à Octobre 2024 au Burkina Faso, par administration territoriale")
plt.xlabel('Mois')
plt.ylabel("Nombre d'événements par admin")
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%b %Y'))
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Filtrer les données pour le Mali
niger_all_data = all_data.loc[all_data['country'] == 'Niger']

# Grouper les données filtrées par 'year', 'month', 'country' et 'admin1'
# et calculer la somme des 'terrorist_events'
niger_terrorist_events_by_admin = niger_all_data.groupby(['year', 'month', 'country', 'admin1'])['terrorist_events'].sum().reset_index()

niger_terrorist_events_by_admin['date'] = pd.to_datetime(niger_terrorist_events_by_admin[['year', 'month']].assign(day=1))
niger_terrorist_events_by_admin.set_index('date', inplace=True)
# Visualisation des événements prédits
number_of_colors = 15

colors = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
             for i in range(number_of_colors)]

dict_color = dict(zip(niger_all_data['admin1'].unique(), colors))
legend_pops = []
plt.figure(figsize=(20, 8))
for admin in niger_all_data['admin1'].unique():
    pred_by_admin = niger_terrorist_events_by_admin[niger_terrorist_events_by_admin['admin1'] == admin]
    niger_pred_by_admin= pred_by_admin[(pred_by_admin['month'] >4) & (pred_by_admin['year'] == 2024)]
    niger_past_by_admin= pred_by_admin[~((pred_by_admin['month'] >4) & (pred_by_admin['year'] == 2024))]
    
    plt.plot(niger_past_by_admin.index, niger_past_by_admin['terrorist_events'], linestyle='-', marker='x', color=dict_color[admin])
    plt.plot(niger_pred_by_admin.index, niger_pred_by_admin['terrorist_events'], linestyle='--', marker='x', color=dict_color[admin])
    legend_pops.append(mpatches.Patch(color=dict_color[admin], label=admin)) 

plt.legend(handles=legend_pops) 

plt.title("Evénements passés et prédits liés à des organisations terroristes, par mois de juin 2021 à Octobre 2024 au Niger, par administration territoriale")
plt.xlabel('Mois')
plt.ylabel("Nombre d'événements par admin")
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%b %Y'))
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:


# Grouper les données filtrées par 'year', 'month', 'country' et 'admin1'
# et calculer la somme des 'terrorist_events'
terrorist_events_by_country = all_data.groupby(['year', 'month', 'country'])['terrorist_events'].sum().reset_index()

terrorist_events_by_country ['date'] = pd.to_datetime(terrorist_events_by_country [['year', 'month']].assign(day=1))
terrorist_events_by_country .set_index('date', inplace=True)
# Visualisation des événements prédits
number_of_colors = 15

colors = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
             for i in range(number_of_colors)]

dict_color = dict(zip(all_data['country'].unique(), colors))
legend_pops = []
plt.figure(figsize=(20, 8))
for country in all_data['country'].unique():
    data_by_country = terrorist_events_by_country[terrorist_events_by_country['country'] == country]
    future_pred_by_country = data_by_country[(data_by_country['month'] >4) & (data_by_country['year'] == 2024)]
    past_by_country = data_by_country[~((data_by_country['month'] >4) & (data_by_country['year'] == 2024))]
    
    plt.plot(past_by_country.index, past_by_country['terrorist_events'], linestyle='-', marker='x', color=dict_color[country])
    plt.plot(future_pred_by_country.index, future_pred_by_country['terrorist_events'], linestyle='--', marker='x', color=dict_color[country])
    legend_pops.append(mpatches.Patch(color=dict_color[country], label=country)) 

plt.legend(handles=legend_pops) 

plt.title("Evénements passés et prédits liés à des organisations terroristes, par mois de juin 2021 à Octobre 2024 par pays")
plt.xlabel('Mois')
plt.ylabel("Nombre d'événements par admin")
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%b %Y'))
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
## VISUALISATION DES PREDICTIONS SUR UNE CARTE
############################################################

##
# D'abord on essaie d'identifier chaque sous-région du Mali

# On charge les fichiers .shp pour les sous-régions du Mali
gdf_mali = gpd.read_file('../references/mali_admin1.shp')


# On trace les sous-régions du Mali avec leurs identifiants
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
gdf_mali.plot(ax=ax, edgecolor='k')

# On ajoute les labels des identifiants sur chaque sous-région
for idx, row in gdf_mali.iterrows():
    plt.annotate(text=idx, xy=(row.geometry.centroid.x, row.geometry.centroid.y),
                 horizontalalignment='center', fontsize=9, color='white')

plt.title('Sous-régions du Mali avec identifiants')
plt.show();

# On affiche les différentes valeurs prises par admin1 afin d'identifier
# chaque sous-région sur la carte et d'associer au nom l'identifiant correspondant
# grâce à une autre carte (source : Wikipédia) du Mali
future_data_mali = future_data[future_data['country'] == 'Mali']
print("Nom des sous-région du Mali :\n", future_data_mali['admin1'].unique())

mapping_mali = {
    'Bamako': 0,
    'Gao': 1,
    'Kayes': 2,
    'Kidal': 3,
    'Koulikoro': 4,
    'Menaka': 1,
    'Mopti': 5,
    'Segou': 6,
    'Sikasso': 7,
    'Tombouctou': 8
}

# On applique le mapping à notre dataframe qui contient les données pour le Mali
future_data_mali_geo = map_regions_to_ids(future_data_mali, mapping_mali)

# On fusionne les données géographiques avec les prédictions
gdf_combined_mali = gdf_mali.merge(future_data_mali_geo, how='left', left_index=True, right_on='region_id')

In [ ]:
##
# Puis on essaie d'identifier chaque sous-région du Burkina Faso

# On charge les fichiers .shp pour les sous-régions du Burkina
gdf_burkina = gpd.read_file('../references/burkina_faso_admin1.shp')


# On trace les sous-régions du Burkina avec leurs identifiants
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
gdf_burkina.plot(ax=ax, edgecolor='k')

# On ajoute les labels des identifiants sur chaque sous-région
for idx, row in gdf_burkina.iterrows():
    plt.annotate(text=idx, xy=(row.geometry.centroid.x, row.geometry.centroid.y),
                 horizontalalignment='center', fontsize=9, color='white')

plt.title('Sous-régions du Burkina Faso avec identifiants')
plt.show();

# On affiche les différentes valeurs prises par admin1 afin d'identifier
# chaque sous-région sur la carte et d'associer au nom l'identifiant correspondant
# grâce à une autre carte (source : Wikipédia) du Burkina
future_data_burkina = future_data[future_data['country'] == 'Burkina Faso']
print("Nom des sous-région du Burkina Faso :\n", future_data_burkina['admin1'].unique())

mapping_burkina = {
    'Boucle du Mouhoun': 0,
    'Cascades': 1,
    'Centre': 2,
    'Centre-Est': 3,
    'Centre-Nord': 4,
    'Centre-Ouest': 5,
    'Centre-Sud': 6,
    'Est': 7,
    'Hauts-Bassins': 8,
    'Nord': 9,
    'Plateau-Central': 10,
    'Sahel': 11,
    'Sud-Ouest': 12
}

# On applique le mapping à notre dataframe qui contient les données pour le Burkina
future_data_burkina_geo = map_regions_to_ids(future_data_burkina, mapping_burkina)

# On fusionne les données géographiques avec les prédictions
gdf_combined_burkina = gdf_burkina.merge(future_data_burkina_geo, how='left', left_index=True, right_on='region_id')

In [ ]:
##
# Ensuite on essaie d'identifier chaque sous-région du Niger

# On charge les fichiers .shp pour les sous-régions du Niger
gdf_niger = gpd.read_file('../references/niger_admin1.shp')


# On trace les sous-régions du Niger avec leurs identifiants
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
gdf_niger.plot(ax=ax, edgecolor='k')

# On ajoute les labels des identifiants sur chaque sous-région
for idx, row in gdf_niger.iterrows():
    plt.annotate(text=idx, xy=(row.geometry.centroid.x, row.geometry.centroid.y),
                 horizontalalignment='center', fontsize=9, color='white')

plt.title('Sous-régions du Niger avec identifiants')
plt.show();

# On affiche les différentes valeurs prises par admin1 afin d'identifier
# chaque sous-région sur la carte et d'associer au nom l'identifiant correspondant
# grâce à une autre carte (source : Wikipédia) du Niger
future_data_niger = future_data[future_data['country'] == 'Niger']
print("Nom des sous-région du Niger :\n", future_data_niger['admin1'].unique())

mapping_niger = {
    'Agadez': 0,
    'Diffa': 1,
    'Dosso': 2,
    'Maradi': 3,
    'Niamey': 4,
    'Tahoua': 5,
    'Tillaberi': 6,
    'Zinder': 7
}

# On applique le mapping à notre dataframe qui contient les données pour le Niger
future_data_niger_geo = map_regions_to_ids(future_data_niger, mapping_niger)

# On fusionne les données géographiques avec les prédictions
gdf_combined_niger = gdf_niger.merge(future_data_niger_geo, how='left', left_index=True, right_on='region_id')

In [ ]:
# On fusionne les GeoDataFrames de tous les pays en un seul GeoDataFrame
gdf_combined = pd.concat([gdf_combined_niger, gdf_combined_mali, gdf_combined_burkina], axis=0, ignore_index=True)

# On vérifie que nos données sont bien du type attendu pour l'affichage sur notre map
gdf_combined['region_id'] = gdf_combined['region_id'].astype(int)
gdf_combined['month'] = gdf_combined['month'].astype(int)
gdf_combined['terrorist_events'] = gdf_combined['terrorist_events'].astype(int)
gdf_combined['terrorist_events_last_6_months'] = gdf_combined['terrorist_events_last_6_months'].astype(int)
gdf_combined['admin1'] = gdf_combined['admin1'].astype(str)

# On retire les colonnes non nécessaires
columns_to_keep = ['geometry', 'region_id', 'month', 'terrorist_events', 'terrorist_events_last_6_months', 'admin1']
gdf_combined = gdf_combined[columns_to_keep]

# On crée une map qui permettra de visualiser pour chaque sous-région des 3 pays
# contenue dans notre GéoDataframe le nombre d'évènements liés à des organisations terroristes
fig = px.choropleth_mapbox(
    gdf_combined,
    geojson=gdf_combined.geometry.__geo_interface__,
    locations=gdf_combined.index,
    color='terrorist_events',
    animation_frame='month',
    hover_data={
        'region_id': False,
        'admin1': True,
        'month': True,
        'terrorist_events': True ,
        'terrorist_events_last_6_months': True
    },
    title="Prédiction des événements impliquant des organisations terroristes pour les 6 prochains mois, par région",
    color_continuous_scale='Plasma',
    mapbox_style="carto-positron",
    center={"lat": 15, "lon": -1.5},
    zoom=5,
    opacity=0.5,
    labels={
        'terrorist_events': 'Événements terroristes',
        'terrorist_events_last_6_months': 'Événements terroristes 6 derniers mois (moyenne)'
        }
)

# On ajuste la taille de la carte
fig.update_layout(
    height=800,
    width=1000
)

# On affiche la carte interactive
fig.show();